In [12]:
import pandas as pd
import numpy as np
import time
import datetime
from collections import defaultdict
import gc

In [14]:
# 列表：店铺id，商品id，用户id，品牌，城市
# 1.计算每个列表的数量和销售等级总和
# 2.计算每个用户在各列表（商品、店铺、品牌）中所占比例

def dolastCount(data):
    
    keys = ['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'item_sales_level']

    for colname in keys:
        print(colname,'starting.....')
        count = data.groupby([colname]).apply(lambda x: x['instance_id'][(x['day']!=7).values].count()).reset_index(name='cnt_'+colname)
        sums = data.groupby([colname]).apply(lambda x: x['item_sales_level'][(x['day']!=7).values].sum()).reset_index(name='sum_'+colname)
        data = pd.merge(data,count, how='left', on=[colname])
        data = pd.merge(data,sums, how='left', on=[colname])
    
    for colname in keys:
        if(colname != 'user_id'):
            print('user_id_', colname, 'starting.....')
            count = data.groupby([colname, 'user_id']).apply(lambda x: x['instance_id'][(x['day']!=7).values].count()).reset_index(name='cnt_'+'user_id_'+colname)
            sums = data.groupby([colname, 'user_id']).apply(lambda x: x['item_sales_level'][(x['day']!=7).values].sum()).reset_index(name='sum_'+'user_id_'+colname)
            data = pd.merge(data,count, how='left', on=[colname, 'user_id'])
            data = pd.merge(data,sums, how='left', on=[colname, 'user_id'])
    
    data['item_user_ratio'] = data['cnt_user_id_item_id']/data['cnt_user_id']
    data['shop_user_ratio'] = data['cnt_user_id_shop_id']/data['cnt_user_id']
    data['brand_user_ratio'] = data['cnt_user_id_item_brand_id']/data['cnt_user_id'] 

    return data

In [18]:
# 1.计算每个小时内商品收藏量、商品浏览量和商品销售量的比率
# 2.判断每个商品的重复次数是否大于2
# 3.计算每个商店和每个商品类别在每天内的查询次数
# 4.计算每个商品在三天内的平均价格
# 5.计算每个商品在三天内的价格差

def doNew(data):

    # collected pv sales 之间的小时比率
    print('collected pv sales 之间的小时比率')
    coll_query = data.groupby(['day', 'hour'], as_index=False)['item_collected_level'].agg({'hour_query_collect': 'sum'})
    pv_query = data.groupby(['day', 'hour'], as_index=False)['item_pv_level'].agg({'hour_query_pv': 'sum'})
    sales_query = data.groupby(['day', 'hour'], as_index=False)['item_sales_level'].agg({'hour_query_sales': 'sum'})
    coll_query = coll_query.merge(pv_query, how='left', on=['day', 'hour'])
    coll_query = coll_query.merge(sales_query, how='left', on=['day', 'hour'])

    coll_query['coll_sales_hour_ratio'] = round(coll_query['hour_query_collect'] / coll_query['hour_query_sales'],5)
    coll_query['coll_sales_hour_ratio'] = coll_query['coll_sales_hour_ratio'] - coll_query['coll_sales_hour_ratio'].min()
    coll_query['pv_sales_hour_ratio'] = round(coll_query['hour_query_pv'] / coll_query['hour_query_sales'], 5)
    coll_query['pv_sales_hour_ratio'] = coll_query['pv_sales_hour_ratio'] - coll_query['pv_sales_hour_ratio'].min()
    
    del coll_query['hour_query_collect']
    del coll_query['hour_query_pv']
    del coll_query['hour_query_sales']

    data = pd.merge(data, coll_query, how='left', on=['day', 'hour'])
    
    #重复次数是否大于2
    print('重复次数是否大于2')
    subset = ['item_brand_id', 'item_id', 'shop_id', 'user_id']
    temp=data.groupby(subset)['is_trade'].count().reset_index()
    temp.columns=['item_brand_id', 'item_id', 'shop_id', 'user_id','large2']
    temp['large2']=1*(temp['large2']>2)
    data = pd.merge(data, temp, how='left', on=subset)
    
    #每天内的查询次数
    shop_query = data.groupby(['shop_id', 'day']).size().reset_index().rename(columns={0: 'shop_id_query_day'})
    category_2_query = data.groupby(['item_category_2', 'day']).size().reset_index().rename(columns={0: 'category_2_query_day'})
    data = pd.merge(data, shop_query, how='left', on=['shop_id', 'day'])
    data = pd.merge(data, category_2_query, how='left', on=['item_category_2', 'day'])

    #商品平均价格差
    print('price diff......')
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==4]
    item_price_4 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_4': 'mean'})
    data = pd.merge(data, item_price_4, how='left', on='item_id')
    del temp
    gc.collect()
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==5]
    item_price_5 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_5': 'mean'})
    data = pd.merge(data, item_price_5, how='left', on='item_id')
    del temp
    gc.collect()
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==6]
    item_price_6 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_6': 'mean'})
    data = pd.merge(data, item_price_6, how='left', on='item_id')
    del temp
    gc.collect()

    data['price_diff_7_6'] = data['item_price_level'] = data['price_6']
    data['price_diff_7_5'] = data['item_price_level'] = data['price_5']
    data['price_diff_7_4'] = data['item_price_level'] = data['price_4']

    del data['price_6']
    del data['price_5']
    del data['price_4']

    return data

In [21]:
# 计算给定数据集中每个用户、商店、商品和商品品牌的历史出现次数。
# 遍历四个特征（user_id、shop_id、item_id、item_brand_id）
# 获取特征的值列表，使用字典存储该特征值出现的次数
# 遍历数据集中的每一行，将该行的特征值作为字典的键，将该键对应的值加1，并赋给名为till_now_cnt的numpy数组
# 将till_now_cnt数组添加到数据集中

def tillNow(data):
    
    for feat in ['user_id', 'shop_id', 'item_id', 'item_brand_id']:
        lists = data[feat].values
        dicts = defaultdict(lambda: 0)
        till_now_cnt = np.zeros(len(data))
        for i in range(len(data)):
            till_now_cnt[i] = dicts[lists[i]]
            dicts[lists[i]] += 1
        if(feat == 'item_brand_id'):
            data[feat.split('_')[1]+'_till_now_cnt'] = till_now_cnt
        else:
            data[feat.split('_')[0]+'_till_now_cnt'] = till_now_cnt

    return data

In [24]:
path = './data/'
    
#train = pd.read_csv(path+'train_all.csv')
#test = pd.read_csv(path+'test_all.csv')
train = pd.read_csv('/Users/user/Desktop/ma/project/dataset/train_all.csv')
test = pd.read_csv('/Users/user/Desktop/ma/project/dataset/test_all.csv')

data = pd.concat([train, test])

print('初始维度:', data.shape)

cols = data.columns.tolist()
keys = ['instance_id', 'day']
for k in keys:
    cols.remove(k)

初始维度: (121617, 35)


In [25]:
data = dolastCount(data)
print('dolastCount:', data.shape)

shop_id starting.....
item_id starting.....
user_id starting.....
item_brand_id starting.....
item_city_id starting.....
item_sales_level starting.....
shop_id starting.....
item_id starting.....
item_brand_id starting.....
item_city_id starting.....
item_sales_level starting.....
dolastCount: (121617, 60)


In [26]:
data = doNew(data)
print('doNew:', data.shape)

collect pv sales 之间的小时比率
重复次数是否大于2
price diff......
item_price_level
item_price_level
item_price_level
doNew: (121617, 68)


In [27]:
data = tillNow(data)
print('tillNow:', data.shape) 

tillNow: (121617, 72)


In [28]:
data = data.drop(cols, axis=1)

# 得到全部训练集
print('经过处理后,最终维度:', data.shape)
print(data.columns.tolist())
#data.to_csv(path+'401_list_till_feat.csv', index=False)
data.to_csv('/Users/user/Desktop/ma/project/dataset/401_list_till_feat_all.csv', index=False)

# 得到7号训练集
data7 = data.loc[data.day==7]
data7 = data7.drop('day', axis=1)
print('经过处理后,7号训练集最终维度:', data7.shape)
print(data7.columns.tolist())
#data7.to_csv(path+'401_list_till_feat7.csv', index=False)
data7.to_csv('/Users/user/Desktop/ma/project/dataset/401_list_till_feat7.csv', index=False)

经过处理后,最终维度: (121617, 39)
['instance_id', 'day', 'cnt_shop_id', 'sum_shop_id', 'cnt_item_id', 'sum_item_id', 'cnt_user_id', 'sum_user_id', 'cnt_item_brand_id', 'sum_item_brand_id', 'cnt_item_city_id', 'sum_item_city_id', 'cnt_item_sales_level', 'sum_item_sales_level', 'cnt_user_id_shop_id', 'sum_user_id_shop_id', 'cnt_user_id_item_id', 'sum_user_id_item_id', 'cnt_user_id_item_brand_id', 'sum_user_id_item_brand_id', 'cnt_user_id_item_city_id', 'sum_user_id_item_city_id', 'cnt_user_id_item_sales_level', 'sum_user_id_item_sales_level', 'item_user_ratio', 'shop_user_ratio', 'brand_user_ratio', 'coll_sales_hour_ratio', 'pv_sales_hour_ratio', 'large2', 'shop_id_query_day', 'category_2_query_day', 'price_diff_7_6', 'price_diff_7_5', 'price_diff_7_4', 'user_till_now_cnt', 'shop_till_now_cnt', 'item_till_now_cnt', 'brand_till_now_cnt']
经过处理后,7号训练集最终维度: (28054, 38)
['instance_id', 'cnt_shop_id', 'sum_shop_id', 'cnt_item_id', 'sum_item_id', 'cnt_user_id', 'sum_user_id', 'cnt_item_brand_id', 'sum_i

In [29]:
data

,instance_id,day,cnt_shop_id,sum_shop_id,cnt_item_id,sum_item_id,cnt_user_id,sum_user_id,cnt_item_brand_id,sum_item_brand_id,...,large2,shop_id_query_day,category_2_query_day,price_diff_7_6,price_diff_7_5,price_diff_7_4,user_till_now_cnt,shop_till_now_cnt,item_till_now_cnt,brand_till_now_cnt
0,7548377597191725106,3,26,280,15,165,1,11,26,280,...,0,5,22,8.0,8.0,8.0,0.0,0.0,0.0,0.0
1,5975219932893529301,6,88,1071,17,161,2,18,17,161,...,0,22,1304,7.0,7.0,NaN,0.0,0.0,0.0,0.0
2,7640080258618597885,3,8,80,6,63,1,10,6,63,...,0,3,909,NaN,NaN,5.0,0.0,0.0,0.0,0.0
3,3961824130940365274,31,41,426,14,134,1,9,43,445,...,0,10,7175,NaN,7.0,7.0,0.0,0.0,0.0,0.0
4,9169996568220897747,1,220,2101,41,451,1,11,220,2101,...,0,34,8076,7.0,7.0,7.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121612,5026363925379972351,7,215,2323,42,504,0,0,105,1227,...,0,65,1895,7.0,7.0,7.0,0.0,279.0,53.0,140.0
121613,45836528932802192,7,81,1268,81,1268,0,0,81,1268,...,0,26,1641,5.0,5.0,5.0,0.0,106.0,106.0,106.0
121614,5774287585256625409,7,22,156,2,14,0,0,22,156,...,0,6,225,NaN,5.0,NaN,0.0,27.0,3.0,27.0
121615,3606899985219981461,7,132,1351,11,109,0,0,132,1351,...,0,12,16909,8.0,NaN,8.0,0.0,143.0,11.0,143.0


In [30]:
data7

,instance_id,cnt_shop_id,sum_shop_id,cnt_item_id,sum_item_id,cnt_user_id,sum_user_id,cnt_item_brand_id,sum_item_brand_id,cnt_item_city_id,...,large2,shop_id_query_day,category_2_query_day,price_diff_7_6,price_diff_7_5,price_diff_7_4,user_till_now_cnt,shop_till_now_cnt,item_till_now_cnt,brand_till_now_cnt
39,5068018660049153081,23,155,0,0,0,0,23,155,2034,...,0,18,16909,NaN,NaN,NaN,0.0,0.0,0.0,0.0
72,2120148911290918063,5,60,5,60,1,10,12148,119522,15392,...,0,1,16909,5.0,NaN,NaN,0.0,0.0,0.0,10.0
95,1413522807664859386,35,466,27,378,2,21,38,496,1326,...,0,24,16909,8.0,8.0,8.0,0.0,0.0,0.0,0.0
161,4763730516779804609,8,73,5,55,0,0,39,345,15392,...,0,5,1895,NaN,NaN,NaN,0.0,0.0,0.0,0.0
170,1786320852452646018,9,92,7,76,0,0,3824,36697,15392,...,0,4,16909,5.0,NaN,5.0,0.0,0.0,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121612,5026363925379972351,215,2323,42,504,0,0,105,1227,723,...,0,65,1895,7.0,7.0,7.0,0.0,279.0,53.0,140.0
121613,45836528932802192,81,1268,81,1268,0,0,81,1268,16590,...,0,26,1641,5.0,5.0,5.0,0.0,106.0,106.0,106.0
121614,5774287585256625409,22,156,2,14,0,0,22,156,1895,...,0,6,225,NaN,5.0,NaN,0.0,27.0,3.0,27.0
121615,3606899985219981461,132,1351,11,109,0,0,132,1351,2380,...,0,12,16909,8.0,NaN,8.0,0.0,143.0,11.0,143.0
